#### Simulation Of Scenarios

In [1]:
## Import Packages
import itertools
import random
import numpy as np

### Torch Packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split # For Subset Selection

### Triple-Gamma-Reg
import tgr as tgr1

In [2]:
## Set Scenario Values
dataset_size = [100] # In Integers; as in N = 100
sparsity_values = [0.02] # In Percentage; as in 10% of coefficients are different from 0
num_of_coefs = [10] # In Integers; as in total number of coefficients is 10

scenarios = list(itertools.product(dataset_size, sparsity_values, num_of_coefs))

In [6]:
def simulateData(scenario):
    n = scenario[0]
    sparsity = scenario[1]
    coef_num = scenario[2]
    
    n_train = int(n*0.9)
    n_test = n - n_train
    snr = 1 # Sound-to-noise-ratio
    
    # Get Explanatory Data drawn from Normal Distribution
    X = torch.randn(n, coef_num)
    
    ## Get True Coefficients for Data generating Process
    coefs_list = int((1-sparsity)*coef_num) * [0] # Create List of Zero Coefficients
    coefs_list.extend(round(random.uniform(-20, -10),5) for _ in range(coef_num - int((1-sparsity)*coef_num))) # Fill up remaining Coeficients with random parameters
    random.shuffle(coefs_list) # Shuffle the List
    coefs_list = torch.tensor(coefs_list)
    
    # Generate Y values; Use Sigma according to Wang et al (2020)
    sigma = np.sqrt((coefs_list.T @ X.T @ X @ coefs_list)/(snr**2*n)).item()
    Y = X @ coefs_list
    
    # Split into train and test
    Y_train = Y[:n_train]
    X_train = X[:n_train,]
    Y_test = Y[n_train:]
    X_test = X[n_train:,]
    
    return Y_test, Y_train, X_test, X_train, coefs_list

In [7]:
## Simulation
torch.manual_seed(1234)
random.seed(12345)
torch.set_printoptions(sci_mode=False, precision=10)

for scenario in scenarios:
    y_test, y_train, x_test, x_train, true_coefs  = simulateData(scenario)
    print(f'Data Set Size: {scenario[0]}; Sparsity: {scenario[1]*100}%; Number of Coefficients: {scenario[2]}')
    
    ### TGR Regression
    testerror_TGR = list()
    
    iterations = 1000
    trained_model2, coefs, loss_of_optimization = tgr1.TripleGammaModel(x_train, y_train, 1, 1, 30, 1, True, num_epochs=iterations) # Covariates, Targets, Penalty, a, c, kappa, normalization=True
    coefficients = trained_model2.linear.weight.detach().numpy()
    print(coefficients[0].tolist())
        
    ### LASSO Regression
    testerror_LASSO = list()
    
    ## Output in Table
    testerror_scenario = [testerror_TGR, testerror_LASSO]

Data Set Size: 100; Sparsity: 2.0%; Number of Coefficients: 10


C:\Users\Lucas Paul\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([90])) that is different to the input size (torch.Size([90, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 252.14590454101562
Epoch [200/1000], Loss: 252.1394805908203
Epoch [300/1000], Loss: 252.13088989257812
Epoch [400/1000], Loss: 252.13943481445312
Epoch [500/1000], Loss: 252.13760375976562
Epoch [600/1000], Loss: 252.13534545898438
Epoch [700/1000], Loss: 252.13699340820312
Epoch [800/1000], Loss: 252.1328125
Epoch [900/1000], Loss: 252.14129638671875
Epoch [1000/1000], Loss: 252.13934326171875
[0.00045080611016601324, -0.0007013143040239811, -0.004835708066821098, -0.0038441491778939962, -0.001191428629681468, 0.0014461460523307323, -0.001939990557730198, -0.00280621531419456, -0.0009467573836445808, -0.00032485031988471746]
